In [1]:
# base imports
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from scipy.stats import shapiro
from sklearn.metrics import r2_score
import logging
logger = logging.getLogger("first_model")

# import helpers
import sys
sys.path.append("../../..")
from utils.data import *
from utils.helpers import *
from utils.setup import Config

In [2]:
book_df, trade_df = get_trade_and_book_by_stock_and_time_id(0, 5)
book_df.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


In [3]:
"""
Logic behind the below function is that if at the end of the period we see a spike in volatility then this is more likely to continue in the next period
than if the volatility spike was observed in the first 20% for example.
"""

from scipy.special import expit

def create_book_features(book_df: pd.DataFrame, rolling_window: int = 20) -> pd.Series:
    features = dict()
    book_df["vwap"] = best_vwap(book_df)

    # some interesting features 
    # features["end_vol_change"] = calc_vol_change(book_df, rolling_window)  # this takes way too long
    features["size_imbalance_change"] = calc_best_size_imbalance(book_df)
    features["prev_realized_vol"] = realized_volatility(book_df["vwap"])
    features["returns_normality"] =  calculate_returns_normality(book_df["vwap"])

    # basic feature
    features["px_movement"] = (book_df["vwap"].iloc[-1] / book_df["vwap"].iloc[0]) - 1
    features["log_return"] = calculate_log_return(book_df)

    # not actual features but useful for later use/identification
    features["stock_id"] = book_df["stock_id"].unique()[0]
    features["time_id"] = book_df["time_id"].unique()[0]

    return pd.Series(features)


def calculate_log_return(book_df):
    return log_return(book_df["vwap"]).dropna().sum()


def calculate_returns_normality(price_series: Union[np.array, pd.Series]):
    log_returns = log_return(price_series).dropna()
    result = shapiro(log_returns.values)
    pval = result.pvalue  # got the pvalue from the test
    
    # want to map the p value to 0 / 1 scale with 1 being normal and 0 being low
    # return sigmoid of 1/pvalue
    # return expit(1/ pval)
    return pval


def calc_best_size_imbalance(book_df: pd.DataFrame, scaling_param: float = 1, average_window: int = 5):

    def calc_imbalance(mini_book_df):
        mini_book_df["ask_imbalance"] = mini_book_df["ask_size1"]**scaling_param / (mini_book_df["ask_size1"]**scaling_param + mini_book_df["bid_size1"]**scaling_param)
        mini_book_df["bid_imbalance"] = mini_book_df["bid_size1"]**scaling_param / (mini_book_df["ask_size1"]**scaling_param + mini_book_df["bid_size1"]**scaling_param)
        return mini_book_df


    mid_book = book_df.iloc[int(len(book_df)/2) -5 : int(len(book_df)/2) + 5].copy()
    mid_book = calc_imbalance(mid_book)

    end_book = book_df.iloc[:-average_window*2].copy()
    end_book = calc_imbalance(end_book)

    end_bid_imbalance, end_ask_imbalance = end_book["bid_imbalance"].mean(), end_book["ask_imbalance"].mean()
    
    # take the max end imbalance and compare to the same corresponding mid imbalance
    if end_bid_imbalance > end_ask_imbalance:
        mid_bid_imbalance = mid_book["bid_imbalance"].mean()
        return (end_bid_imbalance / mid_bid_imbalance) - 1
    else:
        mid_ask_imbalance = mid_book["ask_imbalance"].mean()
        return (end_ask_imbalance / mid_ask_imbalance) - 1


def calc_vol_change(book_df: pd.DataFrame, rolling_window: int = 20):
    assert "vwap" in book_df.columns, "VWAP not calculated."
    vol_df = book_df.rolling(rolling_window).apply(realized_volatility)

    #  calculate the difference between midway in the time-period to the end period
    reference_vol = vol_df.iloc[int(len(vol_df)/2)]["vwap"]
    end_vol = vol_df.iloc[-1]["vwap"]

    return (end_vol / reference_vol) - 1

In [4]:
pd.DataFrame.from_dict(create_book_features(book_df))

,0
size_imbalance_change,2.706509e-01
prev_realized_vol,1.118122e+01
returns_normality,1.326755e-18
px_movement,-9.607197e-01
log_return,-3.237033e+00
stock_id,0.000000e+00
time_id,5.000000e+00


In [5]:
%%timeit
create_book_features(book_df) # without the vol_change feature (660ms with)

6.12 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
def load_and_apply_training_data_per_stock_id(apply_to_book_data: Callable, apply_to_trade_data: Callable, id: int) -> pd.DataFrame:
    """
    Loads in trade and book data per stock and applies the supplied functions to those.
    """

    book, trade = get_trade_and_book_by_stock_and_time_id(id)
    book = pd.DataFrame.from_dict(book.groupby("time_id").apply(apply_to_book_data))
    book["returns_normality"] = np.minimum(1, 1e8*book["returns_normality"]) 
    trade = pd.DataFrame.from_dict(trade.groupby("time_id").apply(apply_to_trade_data))

    logger.info(f"Completed processing for stock_id: {id}")

    return book, trade

In [7]:
# apply this to each observation for all stocks to create the design matrix

book_data, trade_data = load_and_apply_training_data_per_stock_id(create_book_features, lambda x: None, 0)

In [8]:
book_data

,size_imbalance_change,prev_realized_vol,returns_normality,px_movement,log_return,stock_id,time_id
time_id,,,,,,,
5,0.270651,11.181223,1.326755e-10,-0.960720,-3.237033,0.0,5.0
11,0.583067,7.880197,9.084259e-12,1.933403,1.076163,0.0,11.0
16,-0.312492,8.619242,4.904688e-13,0.054887,0.053433,0.0,16.0
31,-0.296297,6.489422,3.187105e-05,2.908627,1.363186,0.0,31.0
62,0.451975,8.067280,3.433926e-08,0.539817,0.431663,0.0,62.0
...,...,...,...,...,...,...,...
32751,-0.426423,10.215432,5.899216e-14,1.994987,1.096940,0.0,32751.0
32753,-0.261149,11.706693,2.866894e-14,1.637879,0.969975,0.0,32753.0
32758,-0.250305,9.067587,1.522091e-05,0.503128,0.407548,0.0,32758.0


In [9]:
from joblib import Parallel, delayed
from typing import List

def get_stock_list():
    return pd.read_csv(Config.data_directory / "train.csv")["stock_id"].unique()


def get_data_set(stock_ids: Union[List[str], None] = None, dataType = 'train'):

    if stock_ids is None:
        stock_ids = get_stock_list()


    features = Parallel(n_jobs=-1)(
        delayed(load_and_apply_training_data_per_stock_id)(create_book_features, lambda x: None, stock_id) 
        for stock_id in stock_ids
    )

    book_features = pd.concat([x[0] for x in features], ignore_index = True)
    trade_features = pd.concat([x[1] for x in features], ignore_index = True)

    return book_features, trade_features

In [37]:
book_data, trade_data = get_data_set()  # takes 11mins to run
book_data.to_pickle("../tmp/first_model_book_data.pickle")  # save to not re-run later

In [10]:
book_data = pd.read_pickle("../tmp/first_model_book_data.pickle")  # save to not re-run later

In [11]:
# match to the target 
book_data["target"] = pd.read_csv(Config.data_directory / "train.csv")["target"]
book_data.head()

,size_imbalance_change,prev_realized_vol,returns_normality,px_movement,log_return,stock_id,time_id,target
0,0.270651,11.181223,1.326755e-10,-0.960720,-3.237033,0.0,5.0,0.004136
1,0.583067,7.880197,9.084259e-12,1.933403,1.076163,0.0,11.0,0.001445
2,-0.312492,8.619242,4.904688e-13,0.054887,0.053433,0.0,16.0,0.002168
3,-0.296297,6.489422,3.187105e-05,2.908627,1.363186,0.0,31.0,0.002195
4,0.451975,8.067280,3.433926e-08,0.539817,0.431663,0.0,62.0,0.001747


In [12]:
# create a training test split
from sklearn.model_selection import train_test_split


X = book_data[["size_imbalance_change", "prev_realized_vol", "returns_normality", "px_movement"]]  # keep only the features
y = book_data["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=Config.random_state, shuffle=False)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((343145, 4), (85787, 4), (343145,), (85787,))

In [13]:
xgb = XGBRegressor(tree_method='hist', random_state = Config.random_state, n_jobs= -1)

In [14]:
%%time
xgb.fit(X_train, y_train)

Wall time: 1.25 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=420,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='hist', validate_parameters=1, verbosity=None)

In [15]:
xgb_preds = xgb.predict(X_train)
R2 = round(r2_score(y_true = y_train, y_pred = xgb_preds), 6)
RMSPE = round(rmspe(y_true = y_train, y_pred = xgb_preds), 6)

In [16]:
print(f'Performance of the naive XGBOOST prediction: R2 score: {R2}, RMSPE: {RMSPE}')

Performance of the naive XGBOOST prediction: R2 score: 0.08346, RMSPE: 1.057514


In [ ]:
# this model has gone wrong somewhere, need to have a look where